## KMeans

In [1]:
import sys
sys.path.insert(1, '/home/eatitall_scripts')
sys.path.insert(1, '/home/root/pctobs/lib/python3.8/site-packages')
import pandas as pd
from extraccion_informacion.clustering import clustering


In [2]:
import numpy as np

In [5]:
csv_path='./../../archivos/datos_con_10_ejemplos_reglas_y_extraccion_entidades.csv'

In [6]:
# Cargamos el archivo CSV
df = pd.read_csv(csv_path)

In [7]:
# df_subsubconjunto=df[['edad','tg','estatina','tas','tad','fg',
#                     'diabetes_con_cv','diabetes_con_hipertension',
#                     'diabetes_con_lipidos','diabetes_con_estatinas','diabetes_mayores_de_65',
#                     'diabetes_mayores_de_65_y_salud_saludable','diabetes_mayores_de_65_y_salud_compleja',
#                     'diabetes_mayores_de_65_y_salud_muy_compleja']]

In [8]:
#Definimos parámetros KMeans
n_clusters=4
init='k-means++'
n_init = 10
max_iter=500
tol=0.0001
random_state= 111
algorithm='elkan'

In [11]:
df.head()

,Unnamed: 0,record_id,prevencion,sexo,f_nac,edad,f_dm,evoldm,cisq,ins_cardiaca,...,diabetes,diabetes_con_cv,diabetes_con_hipertension,diabetes_con_lipidos,diabetes_con_estatinas,diabetes_mayores_de_65,diabetes_mayores_de_65_y_salud_saludable,diabetes_mayores_de_65_y_salud_compleja,diabetes_mayores_de_65_y_salud_muy_compleja,alimentos_encontrados
0,1,56,1.0,1,1952-01-20,71.0,2002-01-20,20.0,0,0.0,...,1,0,0,1,1,1,0,0,0,"carbohidratos refinados, pan blanco, carbohidr..."
1,2,213,1.0,1,1960-06-08,63.0,2011-06-08,11.0,0,0.0,...,1,0,0,1,0,0,0,0,0,"carbohidratos, carbohidratos"
2,3,139-95,1.0,0,1947-09-24,75.0,2013-06-15,8.0,0,0.0,...,1,0,1,0,0,1,0,0,1,"agua, bebidas azucaradas"
3,4,143-14,1.0,0,1985-05-31,37.0,2012-03-06,10.0,0,0.0,...,1,0,0,1,1,0,0,0,0,"grasas, aceite de oliva, aguacates, frutos sec..."
4,5,143-17,1.0,0,1966-12-10,56.0,2000-03-09,22.0,0,0.0,...,1,0,1,1,1,0,0,0,0,carbohidratos


In [12]:
import pandas as pd

# Ajustar la configuración de Pandas para mostrar todas las columnas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)  # Puedes ajustar el ancho según sea necesario

# Suponiendo que 'df' es tu DataFrame
hay_nans = df.isnull().values.any()

if hay_nans:
    print("Hay valores NaN en el DataFrame.")
else:
    print("No hay valores NaN en el DataFrame.")

# Mostrar cuántos NaN hay por columna
nans_por_columna = df.isnull().sum()
print("\nNúmero de NaN por columna:")
print(nans_por_columna)



Hay valores NaN en el DataFrame.

Número de NaN por columna:
Unnamed: 0                                     0
record_id                                      0
prevencion                                     0
sexo                                           0
f_nac                                          0
                                              ..
diabetes_mayores_de_65                         0
diabetes_mayores_de_65_y_salud_saludable       0
diabetes_mayores_de_65_y_salud_compleja        0
diabetes_mayores_de_65_y_salud_muy_compleja    0
alimentos_encontrados                          1
Length: 386, dtype: int64


# Procesar strings

In [14]:
def listar_columnas_string(df):
    columnas_string = [col for col in df.columns if df[col].dtype == 'object']
    return columnas_string

In [16]:
listar_columnas_string(df)

['record_id',
 'f_nac',
 'f_dm',
 'f_inicio',
 'o_hipolipemiante',
 'f_visita_3_6meses',
 'motivo_s_3_6meses',
 'f_mace_d_3_6meses',
 'f_muerte_diferido_3_6meses',
 'fg_30_3_6_meses',
 'fg_60_3_6_meses',
 'cac30_3_6_meses',
 'cac300_3_6_meses',
 'fg_30_6_12meses',
 'fg_60_6_12meses',
 'cac30_6_12meses',
 'cac300_6_12meses',
 'fg_30_12_24meses',
 'fg_60_12_24meses',
 'cac30_12_24meses',
 'cac300_12_24meses',
 'fg_30_24_36meses',
 'fg_60_24_36meses',
 'cac30_24_36meses',
 'cac300_24_36meses',
 'observaciones',
 'alimentos_encontrados']

In [17]:
def listar_columnas_si_no(df):
    columnas_si_no = []
    for col in df.columns:
        if df[col].dtype == 'object':  # Verificar si la columna es de tipo object (potencialmente string)
            valores_unicos = df[col].dropna().unique()  # Obtener valores únicos sin contar NaN
            if set(valores_unicos) <= {'Sí', 'No'}:  # Verificar si todos los valores son 'Sí' o 'No'
                columnas_si_no.append(col)
    return columnas_si_no

In [19]:
listar_columnas_si_no(df)

['fg_30_3_6_meses',
 'fg_60_3_6_meses',
 'cac30_3_6_meses',
 'cac300_3_6_meses',
 'fg_30_6_12meses',
 'fg_60_6_12meses',
 'cac30_6_12meses',
 'cac300_6_12meses',
 'fg_30_12_24meses',
 'fg_60_12_24meses',
 'cac30_12_24meses',
 'cac300_12_24meses',
 'fg_30_24_36meses',
 'fg_60_24_36meses',
 'cac30_24_36meses',
 'cac300_24_36meses']

In [20]:
def convertir_si_no_a_dummy(df, columnas):
    for columna in columnas:
        df[columna] = df[columna].map({'Si': 1, 'No': 0})
    return df

columnas=listar_columnas_si_no(df)
df_dummies=convertir_si_no_a_dummy(df,columnas)

### Eliminamos las fechas y las observaciones de los médicos

In [22]:
def eliminar_elementos(lista_principal, elementos_a_eliminar):
    """
    Devuelve una nueva lista que contiene los elementos de 'lista_principal',
    excluyendo los elementos que se encuentran en 'elementos_a_eliminar'.

    Parámetros:
    lista_principal: Lista de elementos de la cual eliminar elementos.
    elementos_a_eliminar: Lista de elementos a eliminar de la lista principal.
    """
    return [elemento for elemento in lista_principal if elemento not in elementos_a_eliminar]

In [23]:
lista_principal=listar_columnas_string(df)
elementos_a_eliminar=listar_columnas_si_no(df)
variables_string_no_bol=eliminar_elementos(lista_principal, elementos_a_eliminar)

df_con_imc_con_reglas_con_dummy_sin_strings=df_dummies
df_con_imc_con_reglas_con_dummy_sin_strings.drop(variables_string_no_bol, axis=1, inplace=True)

## Clustering

In [28]:
maxClusters=15
inertia=clustering.inertia_of_kmeans(df_con_imc_con_reglas_con_dummy_sin_strings,maxClusters,n_clusters=4,init='k-means++',n_init = 10,max_iter=500,tol=0.0001,random_state= 111,algorithm='elkan')

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').